In [1]:
import sys
sys.path.insert(1, '/Users/kaushiksubramanian/discrete-event-simulator')

In [2]:
import des

# Bank Renege Example
There is a single bank counter. 
Bank customers arrive at random and enter the queue and wait for a random time before leaving the queue.
If they reach the server before they leave, they take a random time to be served before leaving.

## Modeling
1. BankServer - is an extension of the Base Server that generates a random processing time.

2. Bank - is an extension of FIFOServers, that discards customers in the queue who have reneged.

3. Customer - is a class that contains all the informations regarding the customer. It is instantiated when a new customer arrives.

## Modeling Events
1. CustomerArrivalEvent - This is scheduled when a new customer is created. The purpose of this event is to wait till the customer arrives and execute the task upon arrival which are

 . Set up the Renege event
 
 . Request Service from the teller and set up the tasks to execute when the server starts service and ends service

2. CustomerRenegeEvent - Execute renege by setting the 'renege' flag to True if customer has not been served yet.

3. CustomerServiceStartEvent - Marks customer as served by setting the 'served' flag to True

4. CustomerServiceEndEvent - Logs customer service end time

In [8]:
import random

NEW_CUSTOMERS = 5  # Total number of customers
INTERVAL_CUSTOMERS = 10.0  # Generate new customers roughly every x seconds
MIN_PATIENCE = 1 # Min. customer patience
MAX_PATIENCE = 3  # Max. customer patience
TIME_IN_BANK = 12.0 # service time

class BankServer(des.servers.BaseServer):
    """A Server process that defines a random service time."""
    def __init__(self, name, manager, sim):
        super().__init__(name, manager, sim)
    
    def get_processing_time(self):
        return random.expovariate(1.0 / TIME_IN_BANK)
    
class Bank(des.servers.FIFOManagers):
    """A Manager process that overrides to neglect reneged events."""
    def __init__(self, name, sim):
        super().__init__(name, sim)
    
    def pull(self):
        """Override pull to ignore reneged customers."""
        while True:
            _, _, (event_start, event_end) = self.queue.pop()
            if event_start.customer.reneged:
                continue
            return (event_start, event_end)

class Customer:
    """A data class for customer."""
    def __init__(self, name):
        self.name = name
        self.arr_time = None
        self.reneged = False
        self.served = False
        self.reneged_at = None
        self.service_start_at = None
        self.service_end_at = None
    
    def __repr__(self):
        if self.reneged:
            return f'{self.name}; arrived at {self.arr_time};  Reneged at {self.reneged_at}'
        else:
            return f'{self.name}; arrived at {self.arr_time};  Start at {self.service_start_at}; Finished at {self.service_end_at}'
    
class CustomerRenegeEvent(des.simulator.Event):
    """Actions to take upon renege."""
    def __init__(self, customer, sim):
        super().__init__(sim)
        self.customer = customer
        
    def execute(self):
        if not self.customer.served:
            self.customer.reneged = True
            self.customer.reneged_at = self.sim.now

class CustomerServiceStartEvent(des.simulator.Event):
    """Actions to take upon service start."""
    def __init__(self, customer, sim):
        super().__init__(sim)
        self.customer = customer
    
    def execute(self):
        if not self.customer.reneged:
            self.customer.served = True
            self.customer.service_start_at = self.sim.now
    def __repr__(self):
        return self.customer.name
            
class CustomerServiceEndEvent(des.simulator.Event):
    """Actions to take upon service end."""
    def __init__(self, customer, sim):
        super().__init__(sim)
        self.customer = customer
    
    def execute(self):
        self.customer.service_end_at = self.sim.now
    def __repr__(self):
        return self.customer.name
        
        
        
class CustomerArrivalProcess(des.simulator.Process):
    """A process that generates customers."""
    def __init__(self, bank, sim):
        super().__init__('CustomerArrivalProcess', sim)
        self.customer_number = 0
        self.bank_servers = bank
    
    def execute(self):
        customer = Customer(f'Customer#{self.customer_number}')
        self.customer_number += 1
        
        customer.arr_time = self.sim.now
        renege_time = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        renege = CustomerRenegeEvent(customer, self.sim)
        # schedule the renege
        renege.schedule(renege_time, 0)
        service_start = CustomerServiceStartEvent(customer, self.sim)
        service_end = CustomerServiceEndEvent(customer, self.sim)
        # request service
        self.bank_servers.request_service(service_start, service_end)   
        
    def can_continue(self):
        return self.customer_number < NEW_CUSTOMERS
    
    def get_next_event_time(self):
        return random.expovariate(1.0 / INTERVAL_CUSTOMERS)

# The simulation
sim = des.simulator.Simulator('sim')
bank_servers = Bank('Bank', sim)
bank_servers.add_server(BankServer('Teller', bank_servers, sim))
CustomerArrivalProcess(bank_servers, sim).start()
sim.run()
customers

[Customer#0; arrived at 0;  Start at 0; Finished at 3.87885012351057,
 Customer#1; arrived at 21.332754644987965;  Start at 21.332754644987965; Finished at 29.576834478281683,
 Customer#2; arrived at 29.32269601802701;  Start at 29.576834478281683; Finished at 52.71278774290299,
 Customer#3; arrived at 29.948569660779224;  Reneged at 31.405704874989738,
 Customer#4; arrived at 36.53126705581644;  Reneged at 38.04472485792695]

## Car wash example
A car wash consists of two stations at which cars can be washed. Cars enter the car wash and wait in a queue to be served

In [10]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number of machines in the carwash
WASHTIME = 5      # Minutes it takes to clean a car
T_INTER = 7       # Create a car every ~7 minutes
SIM_TIME = 20     # Simulation time in minutes

import pandas as pd

class Car(object):
    
    def __init__(self, name):
        self.name = name
        self.arr_time = None
        self.start_time = None
        self.end_time = None
        self.dirt_removed = None
    
    def __repr__(self):
        name = self.name
        arr = self.arr_time
        strt = self.start_time
        fin = self.end_time
        dirt = self.dirt_removed
        if arr is None:
            return f'{name} not yet arrived'
        if strt is not None:
            if fin is not None:
                return f'{name} arrived at {arr}, service started at {strt}, ended at {fin}, {dirt}% dirt removed'
            else:
                return f'{name} arrived at {arr}, service started at {strt}, service in progress'
        else:
            return f'{name} arrived at {arr}, waiting for service'
    
    def to_dict(self):
        return {'name': self.name,
                'arrival': self.arr_time,
                'service_start': self.start_time,
                'service_finish': self.end_time,
                'dirt_removed': self.dirt_removed}

class Cars():
    # just a holder for the cars
    def __init__(self):
        self.car_num = 0
        self.cars = []
        
    def add_car(self):
        car = Car(f'Car{self.car_num}')
        self.car_num += 1
        self.cars.append(car)
        return car
    
    def to_df(self):
        rows = []
        for car in self.cars:
            dic = car.to_dict()
            if dic['service_start'] is not None:
                rows.append(car.to_dict())
        
        return pd.DataFrame(rows).sort_values(by='arrival')

class Washer(des.servers.BaseServer):
    def __init__(self, name, manager, sim):
        super().__init__(name, manager, sim)
    
    def get_processing_time(self):
        return WASHTIME
    
CarWash = des.servers.FIFOManagers
    
class CarServiceStartEvent(des.simulator.Event):
    def __init__(self, car, sim):
        super().__init__(sim)
        self.car = car
    
    def execute(self):
        self.car.start_time = self.sim.now
        
class CarServiceEndEvent(des.simulator.Event):
    def __init__(self, car, sim):
        super().__init__(sim)
        self.car = car
        
    def execute(self):
        self.car.end_time = self.sim.now
        self.car.dirt_removed = random.randint(50, 99)

        
class CarArrivalProcess(des.simulator.Process):
    def __init__(self, cars, car_wash, sim):
        super().__init__('CarArrivalProcess', sim)
        self.cars = cars
        self.car_wash = car_wash
        
    def can_continue(self):
        """Infinite process."""
        return True
    
    def execute(self):
        # A new car is created and we wait for a random time
        # to generate the next car
        car = self.cars.add_car()
        car.arr_time = self.sim.now
        start_event = CarServiceStartEvent(car, self.sim)
        end_event = CarServiceEndEvent(car, self.sim)
        self.car_wash.request_service(start_event, end_event)
        
    def get_next_event_time(self):
        return random.randint(T_INTER - 2, T_INTER + 2)
    

sim = des.simulator.Simulator('AwesomeSimulator')
car_wash = CarWash('Awesome Car Wash', sim)
car_wash.add_server(Washer('his_excellency', car_wash, sim))
car_wash.add_server(Washer('her majesty', car_wash, sim))
cars = Cars()

# create 4 cars at time 0
for i in range(4):
    # each car is starts an arrival process!
    CarArrivalProcess(cars, car_wash, sim).start()

sim.run(run_until=20)
cars.to_df()

,name,arrival,service_start,service_finish,dirt_removed
0,Car0,0,0.0,0.0,73.0
1,Car1,0,0.0,0.0,94.0
2,Car2,0,0.0,0.0,74.0
3,Car3,0,0.0,5.0,78.0
4,Car4,5,5.0,5.0,72.0
5,Car5,7,7.0,7.0,94.0
6,Car6,7,7.0,10.0,61.0
7,Car7,8,8.0,13.0,86.0
8,Car8,12,12.0,12.0,85.0
9,Car9,13,13.0,17.0,72.0


# Movie Theater
Moviegoers visit a single counter to buy tickets for a movie (random # of tickets for a random choice of movie). If the movie is sold out, they renege (leave the queue)

TODO: To complete this example.

In [ ]:
# import numpy as np
# class Counter(des.servers.Server):
#     def __init__(self, name, sim):
#         super().__init__(name, sim)
    
#      def get_processing_time(self):
#         return 1.0
    
# class MovieTheater(des.servers.FIFOServers):
#     def __init__(self, name, servers, sim):
#         super().__init__(name, servers, sim)
#         self.remaining_tickets = {'Movie1': 50, 'Movie2': 50, 'Movie3': 50}
#         self.sold_out_at = {}
    
#     def pull(self):
#         while True:
#             _, _, (event_start, event_end) = self.queue.pop()
#             movie = event_start.customer.movie_requested
#             if self.remaining_tickets[movie] <= 0:
#                 event_start.customer.reneged = True
#                 event_start.customer.renege_reason = 'Sold Out'
#                 continue
#             return (event_start, event_end)
        
# class Customer():
#     def __init__(self, name):
#         self.name = name
#         self.movie = None
#         self.tickets_requested = None
#         self.reneged = False
#         self.renege_reason = 'n/a'
#         self.arr_time = None
    
#     def to_dict(self):
#         return {'name': self.name,
#                 'movie': self.movie,
#                 'tickets': self.tickets,
#                 'reneged': self.reneged,
#                 'reason': self.renege_reason,
#                 }
        
# class Customers():
    
#     def __init__(self):
#         self.customers = []
#         self.customer_number = 0
        
#     @property    
#     def customer(self):
#         name = f'Customer{self.customer_number}'
#         customer = Customer(name)
#         self.customers.append(customer)
#         self.customer_number += 1
#         return customer

# class CustomerArrivalEvent(des.simulator.Event):
    
#     def __init__(self, customer, movie_theater, customers, sim):
#         super().__init__(sim)
#         self.customer = customer
#         self.movie_theater = movie_theater
#         self.customers = customers
        
#     def execute(self):
#         movie = random.c